# Building Applications Powered by LLMs with LangChain

Prompt engineering is a powerful technique that can help to optimize language models for various applications and research topics. By creating good prompts, the model can be guided to deliver accurate, contextually relevant, and insightful responses.

* [1. Role Prompting](#role)
* [2. Few Shot Prompting](#few_shot)
* [3. Bad Prompt Practices](#bad) 
* [4. Chain Prompting](#chain)
* [5. Well-Structured Prompt Example](#well)
* [99. Resources](#resources)

In [1]:
import os
from keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="role">
    
## 1. Role Prompting
    
</a>

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. 

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

# Create the LLMChain for the prompt
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [3]:
# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"} 

In [4]:
# Run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("Theme:", input_data["theme"])
print("Year:", input_data["year"])
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Journey to the Stars: 3030"


The provided prompt is a good example of role prompting  for several reasons:

- Clear instructions
- Specificity
- Open-ended creativity
- Focus on the task

<hr>
<a class="anchor" id="few_shot">
    
## 2. Few Shot Prompting
    
</a>

In [5]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [6]:
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

In [7]:
formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain.run({})

print("Color: purple")
print("Emotion:", response)

Color: purple
Emotion:  creativity


<hr>
<a class="anchor" id="bad">
    
## 3. Bad Prompt Practices
    
</a>

In [23]:
# An example of a too-vague prompt that provides little context or guidance 
# for the model to generate a meaningful response

from langchain import PromptTemplate

template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="Europe")

'Tell me something about Europe.'

<hr>
<a class="anchor" id="chain">
    
## 4. Chain Prompting
    
</a>

Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

In [24]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [25]:
# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

In [26]:
# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0) 

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: 
Albert Einstein's theory of general relativity is a theory of gravitation that states that the gravitational force between two objects is a result of the curvature of spacetime caused by the presence of mass and energy. It explains the phenomenon of gravity as a result of the warping of space and time by matter and energy.


<hr>
<a class="anchor" id="well">
    
## 5. Well-Structured Prompt Example
    
</a>

Tips to engineer effective prompts:
- Be specific (provide detail and enough context to guide the LLM toward the desired output);
- Force conciseness when needed;
- Encourage the model to explain its reasoning (this way the results can be more accurate, especially for complex tasks);
- Iterate if needed (several refinements may be required to obtain the best possible answer).

In [28]:
from langchain import FewShotPromptTemplate, PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few-shot prompt template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the LLMChain for the user query
response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response:  Practice active listening, be mindful of your body language, and be open to constructive feedback.


<hr>
<a class="anchor" id="resources">
    
## 99. Resources:
    
</a>

- [A Hands-on Guide to Prompt Engineering with ChatGPT and GPT-3](https://dev.to/mmz001/a-hands-on-guide-to-prompt-engineering-with-chatgpt-and-gpt-3-4127)
- [Prompt Engineering Tips and Tricks with GPT-3](https://blog.andrewcantino.com/blog/2021/04/21/prompt-engineering-tips-and-tricks/)
- [Prompt Engineering LLMs with LangChain and W&B](https://wandb.ai/a-sh0ts/langchain_callback_demo/reports/Prompt-Engineering-LLMs-with-LangChain-and-W-B--VmlldzozNjk1NTUw)